class Group:

    owner = models.ForeignKey(settings.AUTH_USER_MODEL,
                              on_delete=models.SET_NULL,
                              null=True,
                              related_name='owned_groups')

if owner (by default will be the creator of the group) leaves the group it will be set to null, and then we will run the resetGroupOwner() to choose the longest serving member in the group



class GroupMembership:


    user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE)
    group = models.ForeignKey(Group, on_delete=models.CASCADE, related_name='members')


A group membership object will be deleted once a user and or their group is deleted:
User deleted -> GroupMemberships for that user deleted
Group deleted -> GroupMemberships for that user deleted


Class TaskTemplate

    creator = models.ForeignKey(
            settings.AUTH_USER_MODEL,
            on_delete=models.SET_NULL,
            null=True,
            blank=True,
            related_name='created_tasks',
            help_text="User who created this task"
            )


    group = models.ForeignKey(
        'Group',
        on_delete=models.CASCADE,
        related_name='tasks',
        help_text='Group this task belongs to'
    )

When the creator of a taskTemplate is deleted the creator field of taskTemplate is setNull.
When the group a taskTemplate is linked to is deleted, that taskTemplate is deleted.

class TaskOccurrence(models.Model):


    template = models.ForeignKey(
        'TaskTemplate',
        on_delete=models.CASCADE,
        related_name='occurrences'
)

    assigned_to = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name='assigned_tasks',
        help_text="Current assignee for the task"
    )

In [ ]:
#TODO decide on how to handle task reassignment when assigned user is deleted / leaves group - in leave group logic need to ensure all tasks assigned to user have been set to null.

When the linked taskTemplate is deleted, the taskOccurrence is deleted.
When the assigned_user is deleted, the assigned_user field is set to null. The task will be reassigned depending on the group reassignment policy - Do I want it ot be auto re-assigned regardless of policy - this task was already assigned to somebody, and they have left so we should reassign automatically?
<li>If the group reassignment rule is on create , then reassign immediatly</li>
<li> run reassignment on next task created or after n days</li>

class Calendar(models.Model):

    user = ForeignKey(
            settings.AUTH_USER_MODEL,
            on_delete=models.CASCADE,
            related_name = 'calendars'
        )

        #credentials
    credential = models.ForeignKey(
        'ExternalCredential',
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name='calendars',
        help_text="OAuth2 credentials used to sync this calendar"
    )

In [ ]:
#TODO do below

When user is deleted all their calenders are also deleted.
When credential is deleted then calendar isn't deleted - needs to be fixed to allow "disconnection of external calendar":
<li>This is to handle the internal calendar - but i will research the impacts of creating a dummy externalCredential for the internal calender and then when user is deleted -> Calendar is deleted ->external cred deleted when user is deleted so no lingering data.  </li>

class Event(models.Model):

    calendar = models.ForeignKey(
        'Calendar',
        on_delete=models.CASCADE,
        related_name='events'
    )

    task_occurrence = models.OneToOneField(
        'TaskOccurrence',
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name='event',
        help_text="Task occurrence that generated this event (if any)."
    )

* When calendar is deleted, all related events are deleted
* When task occurrence is deleted - event is set to null - Due to Event being used to block off calendar slots for user
* an Event is either:
    * a taskOccurrence assigned to a user
    * an event created in their in-app calendar and then pushed to their back_sync_enabled calendars
    * an Event pulled from their external Calendars.

class Message(models.Model):


    group = models.ForeignKey(
        'Group',
        on_delete=models.CASCADE,
        related_name='messages'
    )


    sender = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name='messages'
    )

* When group is deleted, all related messages will be deleted. Groups will be auto deleted as soon as there are no more members.
* When sender is deleted, the messages stay until the group is deleted.
* When sender leaves group, sender will be replaced with null and then the backend will show unknownUser or similar.

class MessageReceipt(models.Model):


    message = models.ForeignKey(
        Message,
        on_delete=models.CASCADE,
        related_name='receipts'
    )
    user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.CASCADE,
        related_name='message_receipts'
    )

In [ ]:
#TODO implement user leaving group - delete all message Receipts
#TODO let users only see messages sent after they join

* When a message is deleted (users will have the option to delete their own message, all messageReceipts for that message will be deleted
* When a user is deleted all their messageReceipts are deleted
* When a user leaves a group - implement Manual deletion of messageReceipts

class TaskSwap:


    task = models.ForeignKey(
        TaskOccurrence,
        on_delete=models.CASCADE,
    )
    from_user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.CASCADE,
        related_name='swaps_initiated'
    )
    to_user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.CASCADE,
        related_name='swaps_accepted'
    )